In [ ]:
import cv2
import numpy as np
from scipy.signal import wiener
import matplotlib.pyplot as plt

# Загрузка и преобразование изображения в полутоновое нормализованное
def load_and_normalize_image(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Загружаем в полутоновом формате
    normalized_image = image / 255.0  # Нормализация
    return normalized_image

# Масочная фильтрация
def mask_filtering(image, kernel):
    return cv2.filter2D(image, -1, kernel)

# Фильтрация с заданной АЧХ
def frequency_filtering(image, threshold=0.1):
    dft = np.fft.fft2(image)
    dft_shift = np.fft.fftshift(dft)
    rows, cols = image.shape
    crow, ccol = rows // 2, cols // 2

    # Создаем маску для пропускания низких частот
    mask = np.zeros((rows, cols), np.uint8)
    mask[crow-int(threshold*rows):crow+int(threshold*rows),
         ccol-int(threshold*cols):ccol+int(threshold*cols)] = 1

    fshift = dft_shift * mask
    f_ishift = np.fft.ifftshift(fshift)
    filtered_image = np.fft.ifft2(f_ishift)
    filtered_image = np.abs(filtered_image)
    return filtered_image

# Медианная фильтрация
def median_filtering(image, noise_density=0.05):
    noisy_image = add_salt_pepper_noise(image, noise_density)
    filtered_image = cv2.medianBlur((noisy_image * 255).astype(np.uint8), 3) / 255.0
    return noisy_image, filtered_image

def add_salt_pepper_noise(image, density=0.05):
    noisy_image = np.copy(image)
    num_salt = np.ceil(density * image.size * 0.5)
    num_pepper = np.ceil(density * image.size * 0.5)

    # Добавляем соль
    coords = [np.random.randint(0, i - 1, int(num_salt)) for i in image.shape]
    noisy_image[coords[0], coords[1]] = 1

    # Добавляем перец
    coords = [np.random.randint(0, i - 1, int(num_pepper)) for i in image.shape]
    noisy_image[coords[0], coords[1]] = 0
    return noisy_image

# Ранговая фильтрация
def rank_filtering(image, rank_type='min'):
    if rank_type == 'min':
        kernel = np.ones((3, 3), np.uint8)
        filtered_image = cv2.erode((image * 255).astype(np.uint8), kernel) / 255.0
    elif rank_type == 'max':
        kernel = np.ones((3, 3), np.uint8)
        filtered_image = cv2.dilate((image * 255).astype(np.uint8), kernel) / 255.0
    else:
        raise ValueError("Unknown rank type. Use 'min' or 'max'.")
    return filtered_image

# Адаптивный фильтр Винера
def wiener_filtering(image, noise_std=0.1):
    noisy_image = image + np.random.normal(0, noise_std, image.shape)
    filtered_image = wiener(noisy_image, (5, 5))
    return noisy_image, filtered_image

# Функция отображения изображений для сравнения
def plot_images(images, titles):
    plt.figure(figsize=(15, 5))
    for i in range(len(images)):
        plt.subplot(1, len(images), i + 1)
        plt.imshow(images[i], cmap='gray')
        plt.title(titles[i])
        plt.axis('off')
    plt.show()

# Основная функция
if __name__ == "__main__":
    image_path = 'your_image_path_here.jpg'  # Замените на путь к вашему изображению
    image = load_and_normalize_image(image_path)
    
    # Масочная фильтрация
    kernel = np.array([[1, 1, 1], [1, -8, 1], [1, 1, 1]], dtype=np.float32)  # Пример лаплассового фильтра
    mask_filtered_image = mask_filtering(image, kernel)

    # Фильтрация с АЧХ
    freq_filtered_image = frequency_filtering(image)

    # Медианная фильтрация
    noisy_median, median_filtered_image = median_filtering(image)

    # Ранговая фильтрация
    rank_filtered_image = rank_filtering(image, rank_type='min')

    # Адаптивный фильтр Винера
    noisy_wiener, wiener_filtered_image = wiener_filtering(image)

    # Отображение результатов
    plot_images([image, mask_filtered_image, freq_filtered_image], 
                ["Original", "Mask Filtered", "Frequency Filtered"])
    plot_images([noisy_median, median_filtered_image], ["Noisy (Salt-Pepper)", "Median Filtered"])
    plot_images([rank_filtered_image], ["Rank Filtered (Min)"])
    plot_images([noisy_wiener, wiener_filtered_image], ["Noisy (Gaussian)", "Wiener Filtered"])
